In [1]:
print("Howdy")

Howdy


In [2]:
import polars as pl

In [3]:
train_data = [
    {
        "action": 1,
        "cost": 2,
        "probability": 0.4,
        "feature1": "a",
        "feature2": "c",
        "feature3": "",
    },
    {
        "action": 3,
        "cost": 0,
        "probability": 0.2,
        "feature1": "b",
        "feature2": "d",
        "feature3": "",
    },
    {
        "action": 4,
        "cost": 1,
        "probability": 0.5,
        "feature1": "a",
        "feature2": "b",
        "feature3": "",
    },
    {
        "action": 2,
        "cost": 1,
        "probability": 0.3,
        "feature1": "a",
        "feature2": "b",
        "feature3": "c",
    },
    {
        "action": 3,
        "cost": 1,
        "probability": 0.7,
        "feature1": "a",
        "feature2": "d",
        "feature3": "",
    },
]

train_df = pl.DataFrame(train_data).with_row_index()
train_df

index,action,cost,probability,feature1,feature2,feature3
u32,i64,i64,f64,str,str,str
0,1,2,0.4,"""a""","""c""",""""""
1,3,0,0.2,"""b""","""d""",""""""
2,4,1,0.5,"""a""","""b""",""""""
3,2,1,0.3,"""a""","""b""","""c"""
4,3,1,0.7,"""a""","""d""",""""""


In [27]:
test_data = [
    {"feature1": "b", "feature2": "c", "feature3": ""},
    {"feature1": "a", "feature2": "", "feature3": "b"},
    {"feature1": "b", "feature2": "b", "feature3": "d"},
    {"feature1": "a", "feature2": "", "feature3": "b"},
]

test_df = pl.DataFrame(test_data).with_row_index()

test_df

index,feature1,feature2,feature3
u32,str,str,str
0,"""b""","""c""",""""""
1,"""a""","""""","""b"""
2,"""b""","""b""","""d"""
3,"""a""","""""","""b"""


In [5]:
import vowpal_wabbit_next as vw

In [28]:
workspace = vw.Workspace(["--cb", "4"])
parser = vw.TextFormatParser(workspace)

In [29]:
for row in train_df.iter_rows(named=True):
    action = row["action"]
    cost = row["cost"]
    probability = row["probability"]
    feature1 = row["feature1"]
    feature2 = row["feature2"]
    feature3 = row["feature3"]

    # Construct the example in the required vw format.
    learn_example = parser.parse_line(
        str(action)
        + ":"
        + str(cost)
        + ":"
        + str(probability)
        + " | "
        + str(feature1)
        + " "
        + str(feature2)
        + " "
        + str(feature3)
    )

    # Here we do the actual learning.
    workspace.learn_one(learn_example)

In [30]:
for row in test_df.iter_rows(named=True):
    feature1 = row["feature1"]
    feature2 = row["feature2"]
    feature3 = row["feature3"]

    test_example = parser.parse_line("| " + str(feature1) + " " + str(feature2) + " " + str(feature3))

    choice = workspace.predict_one(test_example)
    print(row["index"], choice)

0 3
1 3
2 1
3 3
